Project Baseline Model

In [11]:
from transformers import AutoModelForTokenClassification, pipeline, RobertaTokenizerFast, Trainer, TextClassificationPipeline
#import numpy as np
#import tqdm as notebook_tqdm
#from datasets import Dataset, DatasetDict
#import torch
import pandas as pd
import pickle

model_link = 'deepset/roberta-base-squad2'
data_source = 'tokenized_output.csv'

Read in raw data (to get sentences)

In [12]:
data = pd.read_csv(data_source)
data

# i think the labels that have been given here may just be the sentiment labels to every word in the review
# SO we may need to readjust the creation of the data or can look at it here to re label the tokens (if they're not -100 'the extra subwords')

,input_ids,attention_mask,labels
0,"[0, 509, 9, 5, 97, 34910, 34, 2801, 14, 71, 24...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[0, 83, 4613, 410, 931, 4, 20, 9293, 9205, 16,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 1, 1, 1, 1, -100, 1, 1, 1, 1, 1, 1, -10..."
2,"[0, 38, 802, 42, 21, 10, 4613, 169, 7, 1930, 8...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[0, 23381, 89, 18, 10, 284, 147, 10, 410, 2143...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, -100, ..."
4,"[0, 5106, 1334, 21129, 118, 18, 22, 16587, 11,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 1, -100, 1, -100, -100, 1, -100, 1, 1, ..."
...,...,...,...
49995,"[0, 38, 802, 42, 1569, 222, 10, 159, 235, 205,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -100, 1, ..."
49996,"[0, 5654, 6197, 6, 1099, 6054, 6, 1099, 3501, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, -100, 0, 0, -100, 0, 0, -100, 0, ..."
49997,"[0, 38, 524, 10, 4019, 5850, 11, 2242, 4306, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, -100, -100, 0, 0, ..."
49998,"[0, 38, 437, 164, 7, 33, 7, 11967, 19, 5, 986,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [52]:
def create_newer_label_list(string_of_label_list):

    an_actual_list = string_of_label_list.split(', ')

    newer_label_list = ['-100' if x== '[-100' or x=='-100]' else x for x in an_actual_list]

    return newer_label_list

Model Load

In [14]:
baseline_model = AutoModelForTokenClassification.from_pretrained(pretrained_model_name_or_path='../project/baseline_model/output_trainer/checkpoint-4704', use_safetensors = True )
tokenizer = RobertaTokenizerFast.from_pretrained(model_link, use_fast=True, add_prefix_space=True)

In [15]:
with open('../project/baseline_model/lab2idx', 'rb') as file:
    lab2idx = pickle.load(file)

with open('../project/baseline_model/idx2lab', 'rb') as file:
    idx2lab = pickle.load(file)


In [16]:
print(lab2idx)
print(idx2lab)

{'I-PER': 0, 'B-PER': 1, 'B-LOC': 2, 'B-ORG': 3, 'I-ORG': 4, 'I-LOC': 5, 'O': 6}
{0: 'I-PER', 1: 'B-PER', 2: 'B-LOC', 3: 'B-ORG', 4: 'I-ORG', 5: 'I-LOC', 6: 'O'}


In [ ]:
#def convert_int_to_labels(preds):
#    logits, labels = preds
#    predictions = np.argmax(logits, axis = -1)
#    true_labels = [[idx2lab[label] for label in label_sequence if label != -100] for label_sequence in labels]
#    true_predictions = [[idx2lab[pred] for pred, label in zip(preds_sequence, labels_sequence) if label != -100] for preds_sequence, labels_sequence in zip(predictions, labels)]
#    return true_labels, true_predictions

In [19]:
#token_classifier_pipeline = TextClassificationPipeline(model = baseline_model, tokenizer = tokenizer, )

In [20]:
a_review = "This says something about a restaurant in Japan that Britney Spears visited in June"

In [21]:
classifier = pipeline('ner', model = baseline_model, tokenizer=tokenizer)

a_review_prediction = classifier(a_review)

Device set to use cpu


In [22]:
a_review_prediction[0]

{'entity': 'B-LOC',
 'score': np.float32(0.99369836),
 'index': 8,
 'word': 'ĠJapan',
 'start': 42,
 'end': 47}

In [23]:
review_df = pd.read_csv("imdb_dataset.csv")
review_df["review"] = review_df["review"].str.replace("<br /><br />", " ", regex=False)
review_df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming te...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [25]:
review_df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me. The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word. It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away. I would say the main appeal of the show is due to the fact that it goes where other shows

In [54]:
# for each review in the df
#apply the NE prediction on it
#save as a label list of length equivalent to the og review
#With the chosen label at the same index as the og word
#currently the output word/sub-word is according to the sub-word review length, so i wanna shorten this

output_new_labels_data = []

# for each review in the df
for review_index in range(len(review_df)):

    current_label_list = create_newer_label_list(string_of_label_list= data['labels'][review_index])
    review = review_df['review'][review_index]

#apply the NE prediction on it
    label_predictions = classifier(review)
#save as a label list of length equivalent to the og review


    for label_prediction in label_predictions:
        required_index_of_label = label_prediction['index']
    #With the chosen label at the same index as the og word


        if current_label_list[required_index_of_label] != -100: #pretty sure this is where i need the label/id converter
            label_replacement = lab2idx[label_prediction['entity']]

            print(type(label_replacement))
            print(label_prediction)
            print(type(current_label_list[required_index_of_label]))
            
            current_label_list[required_index_of_label] = label_replacement

    output_new_labels_data.append[current_label_list]

    

    


<class 'int'>
{'entity': 'B-ORG', 'score': np.float32(0.6772501), 'index': 13, 'word': 'ĠOz', 'start': 68, 'end': 70}
<class 'str'>
<class 'int'>
{'entity': 'B-PER', 'score': np.float32(0.38497344), 'index': 40, 'word': 'ĠOz', 'start': 192, 'end': 194}
<class 'str'>
<class 'int'>
{'entity': 'B-ORG', 'score': np.float32(0.8046325), 'index': 106, 'word': 'ĠO', 'start': 476, 'end': 477}
<class 'str'>
<class 'int'>
{'entity': 'B-ORG', 'score': np.float32(0.73482704), 'index': 116, 'word': 'ĠOswald', 'start': 516, 'end': 522}
<class 'str'>


IndexError: list index out of range

We can't do any metric evaluation on the NE tagging here as there's no ground truth, but Rob gave our model an ~85% span_f1 score